## Python refresher workshop 1

During this workshop (spanning four weeks), we will build a few different syllabification systems, i.e. models which can split words into sequences of syllables:

```
aɪ s l ə n d ə  ->  aɪ s . l ə n . d ə
```

In this first meeting, we will focus on data processing and evaluation. We'll also implement a  baseline syllabification algorithm. A baseline model is essentially a simple model that acts as a reference in a machine learning project. Our baseline will simply chunk the input into segments of two characters (with an optional single character segment at the end):

```
aɪ s l ə n d ə  ->  aɪ s . l ə . n d . ə
t͡ʃ aɪ n ə t aʊ n z   t͡ʃ aɪ . n ə . t aʊ . n z
```

This is obviously not a great result. For example, `n d` is a pretty weird syllable.  However, this will give us a baseline performance that we can compare to when we start developing more complex syllabification systems. Any reasonable syllabifier needs to be able to beat this simple baseline.

The idea is to work on the exercises in pairs or small teams (though you can also work individually if you prefer). For people with no python experience, it's a good idea to team up with someone who is a bit more experienced.  

The following python lectures on Canvas can be useful:

* [String handling](https://canvas.ubc.ca/courses/65386/files/9375671?module_item_id=2255551)
* [Loops](https://canvas.ubc.ca/courses/65386/files/9375677?module_item_id=2255559)
* [Reading from files](https://canvas.ubc.ca/courses/65386/files/9375679?module_item_id=2255563)

These materials should take you pretty far, but googling can also be useful. You are also very welcome to ask each other, Miikka and Roger for help. 

There are hints in the notebook which you can view by clicking on the hint. It's probably a good idea to try first and look at the hint if you need additional help.

If you feel that you have no idea how to get started, let Miikka and Roger know. We can discuss the exercises together.  

### 0. Preparation

If you're using [Google Colab](https://colab.research.google.com/?utm_source=scs-index), first upload this notebook (under the "Upload" tab). Once you're in the Colab, navigate to the "Files" panel to the left, create a new folder called "data" (by right-clicking and selecting "New folder"), and then upload the train, dev, and test files into the newly-created data folder (by right-clicking on the "data" folder and hitting "Upload"). 

### 1. Reading the data

We are given data files in the following format:

```
'd          d                    d
Bulls       b ʊ l z              b ʊ l z
Chinatowns  t͡ʃ aɪ n ə t aʊ n z   t͡ʃ aɪ . n ə . t aʊ n z
I'll        aɪ l                 aɪ l
Icelander   aɪ s l ə n d ə       aɪ s . l ə n . d ə
```

This is a [TSV](https://www.loc.gov/preservation/digital/formats/fdd/fdd000533.shtml) file which contains three tabulator-separated (```'\t'```) columns:

1. Orthographic word
2. IPA transcription
3. Syllabified IPA transcription

All IPA symbols are separated by single spaces and syllable boundaries are marked by a `.`

We recommend first writing a function `read_line()`, which takes a line (i.e. string consisting of three tab-separated fields) as input, e.g.:

```
"Chinatowns  t͡ʃ aɪ n ə t aʊ n z   t͡ʃ aɪ . n ə . t aʊ n z"
```

and converts it into a Python dictionary having the following format:

```
{"orth": "Chinatowns",
 "ipa": ["t͡ʃ", "aɪ", "n", "ə", "t", "aʊ", "n", "z"],
 "syll": [(["t͡ʃ", "aɪ"], 0, 2), (["n", "ə"], 2, 4), (["t", "aʊ", "n", "z"], 4, 8)]}
```

Apart from `syll`, the fields are pretty self-explanatory. In the `syll` field, we've got a list of tuples representing each syllable, its start index and end index (which is 1 + the index of its final character). E.g. the syllable `"n", "ə"`, in the example above, starts at index 2 and ends at index 4:

```
IPA:   "t͡ʃ", "aɪ", "n", "ə", "t", "aʊ", "n", "z"
index:  0     1     2    3    4    5     6    7
```

After implementing the function `read_line()`, you can implement a function `read_data()`, which reads a file into a list of dictionaries. 

Use `read_data()` to read the training, development and test data and store the result in variables `train_data`, `dev_data` and `test_data`. 

<details>
    <summary style="font-weight: bold;">Click here to see the first hint</summary>
    To create the "syll" list, you need to loop through the syllabified IPA transcription and track the index of each IPA symbol (There is a function that allows you to get index while looping). Note that because of the presence of the syllable boundary ".", the raw index of an IPA symbol needs to be modified to get the correct index.
</details>

<details>
    <summary style="font-weight: bold;">Click here to see the second hint</summary>
    You'll need to initialize two variables before looping through the transcription. One variable is used to track the start index for each variable, while the other variable is used to track how many "." you have encountered. The latter variable can then be used to derive the correct start and end indices.
</details>

<details>
    <summary style="font-weight: bold;">Click here to see the third hint</summary>
    Here is the skeleton for one possible way to solve the problem:
    <code>
    for index, symbol in IPAs:
        case 1: if symbol is "." (i.e., you reach the end of a syllable)
            add the current syllable to syll list
            update the two variables that track start index and # of "." seen
        case 2: if we reach the end of IPAs (last syllable)
            add the last syllable to syll list
        case 3: "else" condition
            add the symbol to the current syllable
    </code>
</details>

In [ ]:
# your code here

### 2. Baseline

Today we will implement a very trivial baseline syllabifier function `baseline()`. It contains no phonological insight. Instead, it simply chops the input word into "syllables" of length 2. E.g. given the input:

```
t͡ʃ aɪ n ə t aʊ n z
```

the baseline function would syllabify:

```
t͡ʃ aɪ . n ə . t aʊ . n z
```

**Note:** If the input contains an odd number of IPA symbols, then the final symbol should constitute a singleton syllable. E.g, `aɪ s l ə n d ə -> aɪ s . l ə . n d . ə`. 

Given the input string:

```
["t͡ʃ", "aɪ", "n", "ə", "t", "aʊ", "n", "z"]
```

`baseline()` should return:

```
[(["t͡ʃ", "aɪ"], 0, 2), (["n", "ə"], 2, 4), (["t", "aʊ"], 4, 6), (["n", "z"], 6, 8)]
```

<details>
    <summary style="font-weight: bold;">Click here to see the first hint</summary>
    You can use the index value to see if you need to insert a syllable boundary ".".
</details>

<details>
    <summary style="font-weight: bold;">Click here to see the second hint</summary>
    More specifically, whether to insert a boundary is determined by whether the current index is odd or even. You can use the remainder of a 2-division to see if the index is odd or even. The modulo operator <code>%</code> will be handy here. Note that you do NOT need to insert a syllable boundary "." after the last syllable.
</details>

In [3]:
# your code here

### 3. Evaluation

We will evaluate the performance of the baseline system using [F1-score](https://deepai.org/machine-learning-glossary-and-terms/f-score).

![](https://upload.wikimedia.org/wikipedia/commons/2/26/Precisionrecall.svg)

E.g. given gold standard syllabified strings:

```
[(["t͡ʃ", "aɪ"], 0, 2), (["n", "ə"], 2, 4), (["t", "aʊ", "n", "z"], 4, 8)]
[(["aɪ", "s"], 0, 2), (["l", "ə", "n"], 2, 5), (["d", "ə"], 5, 7), (["s"], 7, 8)]
```

and a baseline system output:

```
[(["t͡ʃ", "aɪ"], 0, 2), (["n", "ə"], 2, 4), (["t", "aʊ"], 4, 6), (["n", "z"], 6, 8)]
[(["aɪ", "s"], 0, 2), (["l", "ə"], 2, 4), (["n", "d"], 4, 6), (["ə", "s"], 7, 8)]
```

we have 3 true positives: 

```
["t͡ʃ", "aɪ"], ["n", "ə"], ["aɪ", "s"] 
```
and 5 false positives:

```
["t", "aʊ"], ["n", "z"], ["l", "ə"], ["n", "d"], ["ə", "s"]
``` 

and 4 false negatives: 

```
["t", "aʊ", "n", "z"], ["l", "ə", "n"], ["d", "ə"], ["s"]
``` 

This results in precision: 

$$P = \frac{\text{true pos}}{(\text{true pos} + \text{false pos})} = 3/8$$ 

and recall: 

$$R = \frac{\text{true pos}}{(\text{true pos} + \text{false neg})} = 3/7$$ 

giving F1-score: 

$$F_1 = 2 * P * R / (P + R) = 0.4 $$

You should implement a function `evaluate()` which takes two lists as input: (1) a list of system output syllabified strings, and (2) a list of gold standard syllabified strings. The function then computes the F1-score and returns it. 

**Note:** You should sum up the true positive, false positive and false negative scores over the entire dataset before computing F1-score.

<details>
    <summary style="font-weight: bold;">Click here to see the first hint</summary>
    Note that you need to compare the corresponding gold standard and system output. That is, you shouldn't collect the syllables from all gold standards into a giant gold set, collect the syllables from all system outputs into a giant system set, and calculate precision and recall use these two sets.
</details>

<details>
    <summary style="font-weight: bold;">Click here to see the second hint</summary>
    The set operations like intersection and difference will be useful here to calculate true positive, false positive, and false negative.
</details>

In [4]:
# your code here